In [23]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [8]:
df = pd.read_csv('C:/Users/DELL/Desktop/finalyearproject/allitems.csv', header=None)


C:\Users\DELL\AppData\Local\Temp\ipykernel_1320\3942872096.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/DELL/Desktop/finalyearproject/allitems.csv', header=None)


In [9]:
df.head()

,0,1,2,3,4,5,6
0,CVE-1999-0001,Candidate,ip_input.c in BSD-derived TCP/IP implementatio...,BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...
1,CVE-1999-0002,Entry,Buffer overflow in NFS mountd gives root acces...,BID:121 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
2,CVE-1999-0003,Entry,Execute commands as root via buffer overflow i...,BID:122 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
3,CVE-1999-0004,Candidate,"MIME buffer overflow in email clients, e.g. So...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i..."
4,CVE-1999-0005,Entry,Arbitrary command execution via IMAP buffer ov...,BID:130 | URL:http://www.securityfocus.com...,NaN,NaN,NaN


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286560 entries, 0 to 286559
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       286560 non-null  object
 1   1       286560 non-null  object
 2   2       286560 non-null  object
 3   3       212787 non-null  object
 4   4       283507 non-null  object
 5   5       283507 non-null  object
 6   6       1855 non-null    object
dtypes: object(7)
memory usage: 15.3+ MB


In [4]:
dataset = pd.read_csv('C:/Users/DELL/Desktop/finalyearproject/CVEFixes.csv')


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31194 entries, 0 to 31193
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   code      31160 non-null  object
 1   language  31194 non-null  object
 2   safety    31194 non-null  object
dtypes: object(3)
memory usage: 731.2+ KB


In [6]:
dataset.head()

,code,language,safety
0,package org.bouncycastle.jcajce.provider.asymm...,java,vulnerable
1,<?php\n\n\n\n/**\n\n * ownCloud - user_ldap\n\...,php,vulnerable
2,#!/usr/bin/env python\n\nfrom __future__ impor...,py,safe
3,/* -*- c-basic-offset: 8 -*-\n\n rdesktop: A...,c,safe
4,<!DOCTYPE html>\n\n<html>\n\n <head>\n\n ...,html,safe


In [91]:
description = df.iloc[:, 2]

In [15]:
description.head()

0    ip_input.c in BSD-derived TCP/IP implementatio...
1    Buffer overflow in NFS mountd gives root acces...
2    Execute commands as root via buffer overflow i...
3    MIME buffer overflow in email clients, e.g. So...
4    Arbitrary command execution via IMAP buffer ov...
Name: 2, dtype: object

In [14]:
description.isnull().sum()

0

In [29]:
nltk.download('punkt')  #Pretrained model to break a sentence into a text list and retain each semantic unit as a component


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [64]:
def word_segmentation(text):
    words = word_tokenize(text)
    return words


In [65]:

temp= word_segmentation("An information disclosure vulnerability exists when the Windows TCP/IP stack improperly handles fragmented IP packets, aka 'Windows TCP/IP Information Disclosure Vulnerability'")
print(temp[0])

An


In [92]:
description = description.apply(word_segmentation)

In [77]:
description.head()

0    [ip_input.c, in, BSD-derived, TCP/IP, implemen...
1    [Buffer, overflow, in, NFS, mountd, gives, roo...
2    [Execute, commands, as, root, via, buffer, ove...
3    [MIME, buffer, overflow, in, email, clients, ,...
4    [Arbitrary, command, execution, via, IMAP, buf...
Name: 2, dtype: object

In [98]:
# word normalization - 1
# removing numbers and decapitalize first letter of sentence

def normalization1(list1):
    
    temp = []
    for word in list1:
        if not word.isnumeric():
            temp.append(word)
    if not temp[0][1:].isupper(): #if the entire word isn't capital
        temp[0] = temp[0].lower()
    return temp

In [99]:
print(normalization1(description[1]))

['buffer', 'overflow', 'in', 'NFS', 'mountd', 'gives', 'root', 'access', 'to', 'remote', 'attackers', ',', 'mostly', 'in', 'Linux', 'systems', '.']


In [100]:
description = description.apply(normalization1)

In [101]:
description.head()

0    [ip_input.c, in, BSD-derived, TCP/IP, implemen...
1    [buffer, overflow, in, NFS, mountd, gives, roo...
2    [execute, commands, as, root, via, buffer, ove...
3    [MIME, buffer, overflow, in, email, clients, ,...
4    [arbitrary, command, execution, via, IMAP, buf...
Name: 2, dtype: object

In [126]:
# word lemmatization

nltk.download('averaged_perceptron_tagger')
from lemminflect import getAllLemmas, getInflection

def lemmatization(list1):
    pos_tags = nltk.pos_tag(list1)

# Apply transformations based on part-of-speech
    transformed_sentence = []
    for word, pos_tag in pos_tags:
        if pos_tag.startswith('N'):  # Noun
            lemmatizations = getAllLemmas(word, upos='NOUN').get('NOUN', [])
            transformed_word = lemmatizations[0] if lemmatizations else word
        elif pos_tag.startswith('V'):  # Verb
            lemmatizations = getAllLemmas(word, upos='VERB').get('VERB', [])
            transformed_word = lemmatizations[0] if lemmatizations else word
        elif pos_tag.startswith('J'):  # Adjective
            lemmatizations = getAllLemmas(word, upos='ADJ').get('ADJ', [])
            transformed_word = lemmatizations[0] if lemmatizations else word
        elif pos_tag.startswith('R'):  # Adverb
            lemmatizations = getAllLemmas(word, upos='ADV').get('ADV', [])
            transformed_word = lemmatizations[0] if lemmatizations else word
        else:
            transformed_word = word  # Keep other parts of speech unchanged

        transformed_sentence.append(transformed_word)

    return transformed_sentence

    


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [127]:
print(lemmatization(description[1]))

['buffer', 'overflow', 'in', 'NFS', 'mountd', 'give', 'root', 'access', 'to', 'remote', 'attacker', ',', 'mostly', 'in', 'Linux', 'system', '.']


In [129]:
description = description.apply(lemmatization)

In [130]:
description.head()

0    [ip_input.c, in, BSD-derived, TCP/IP, implemen...
1    [buffer, overflow, in, NFS, mountd, give, root...
2    [execute, command, as, root, via, buffer, over...
3    [MIME, buffer, overflow, in, email, client, ,,...
4    [arbitrary, command, execution, via, IMAP, buf...
Name: 2, dtype: object

In [131]:
# noise removal

# creating set for reserved words
nltk.download('words')
nltk_words = set(nltk.corpus.words.words())

# Extended set of C/C++ keywords and common functions/commands
cpp_keywords = set([
    # C++ Keywords
    "auto", "const", "double", "float", "int", "short", "struct", "unsigned", "break",
    "continue", "else", "for", "long", "signed", "switch", "void", "case", "default",
    "enum", "goto", "register", "sizeof", "typedef", "volatile", "char", "do", "extern",
    "if", "return", "static", "union", "while", "class", "delete", "friend", "inline",
    "new", "template", "this", "virtual", "bool", "explicit", "mutable", "operator",
    "throw", "typename", "wchar_t", "const_cast", "dynamic_cast", "reinterpret_cast",
    "static_cast",

    # C Standard Library Functions
    "printf", "scanf", "getchar", "putchar", "malloc", "free", "calloc", "realloc",

    # C++ Standard Library Functions
    "cin", "cout", "cerr", "clog", "endl", "include", "iostream", "iomanip", "fstream",
    "ifstream", "ofstream", "string", "vector", "map", "set", "list", "algorithm",
    "iterator", "sort", "max", "min", "strlen", "strcmp", "strcat", "strcpy", "toupper",
    "tolower", "stringstream", "tuple", "make_tuple",

    # Common C Commands
    "#define", "#include", "main", "sizeof", "NULL", "NULL_PTR",
])

reserved_vocabulary = nltk_words.union(cpp_keywords)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [133]:
def noise_removal(list1):
    temp =[]
    for word in list1:
        if word=='/' or word=='.': #remove file paths
            continue 
        elif not word.islower(): #take words having uppercase letter(s)
            temp.append(word)
        elif word in reserved_vocabulary: # words from reserved vocabulary
            temp.append(word)
        else: #check if word combination in valid words
            for i in range(1, len(word)):
                prefix = word[:i]
                suffix = word[i:]

                # Check if both prefix and suffix are in the reserved vocabulary
                if prefix in reserved_vocabulary and suffix in reserved_vocabulary:
                    temp.append(word) 
    return temp


In [134]:
print(noise_removal(description[1]))

['buffer', 'overflow', 'in', 'NFS', 'mountd', 'give', 'root', 'access', 'to', 'remote', 'attacker', ',', 'mostly', 'in', 'Linux', 'system']


In [135]:
description = description.apply(noise_removal)

In [136]:
description.head()

0    [in, BSD-derived, TCP/IP, implementation, allo...
1    [buffer, overflow, in, NFS, mountd, give, root...
2    [execute, command, as, root, via, buffer, over...
3    [MIME, buffer, overflow, in, email, email, cli...
4    [arbitrary, command, execution, via, IMAP, buf...
Name: 2, dtype: object

In [140]:
# stop-word filtering
nltk.download('stopwords')
def remove_stopwords(list1):
    return [i for i in list1 if i not in stopwords.words('english')]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [141]:
print(remove_stopwords(description[1]))

['buffer', 'overflow', 'NFS', 'mountd', 'give', 'root', 'access', 'remote', 'attacker', ',', 'mostly', 'Linux', 'system']


In [143]:
description = description.apply(remove_stopwords)

In [144]:
description.head()

0    [BSD-derived, TCP/IP, implementation, allow, r...
1    [buffer, overflow, NFS, mountd, give, root, ac...
2    [execute, command, root, via, buffer, overflow...
3    [MIME, buffer, overflow, email, email, client,...
4    [arbitrary, command, execution, via, IMAP, buf...
Name: 2, dtype: object

In [146]:
# word normalizaton 2

#remove punctuations, symbols, special characters and convert all uppercase letters into lowercase

def normalization2(list1):
    temp=[]
    for w in list1:
        if w.isalpha():
            temp.append(w.lower())
    return temp

In [147]:
print(normalization2(description[1]))

['buffer', 'overflow', 'nfs', 'mountd', 'give', 'root', 'access', 'remote', 'attacker', 'mostly', 'linux', 'system']


In [148]:
description = description.apply(normalization2)

In [149]:
description.head()

0    [implementation, allow, remote, attacker, caus...
1    [buffer, overflow, nfs, mountd, give, root, ac...
2    [execute, command, root, via, buffer, overflow...
3    [mime, buffer, overflow, email, email, client,...
4    [arbitrary, command, execution, via, imap, buf...
Name: 2, dtype: object